<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-11 09:24:17
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 8.08 K (0.06%)
Current PnL: -12.91 L (-8.99%)
CY Booked + Current PnL: -5.95 L (-4.14%)
-------------------
Total profit:  4.99 L
Total loss:  -17.89 L
-------------------
Total Booked + Current PnL: 21.03 L (18.05%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.07%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.44 L (52.07%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.96%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-221.81,H-SC,93.75,87867.0,-7449.0,7486.0,0.40,-7.82,8.52,0.04,157.0,-1.00,0.64,73.31,OX40N,NTT,MISC
52,JPPOWER,18.73,26.20,-7.22,L-SC,100.00,186372.0,44530.0,12040.0,6.86,31.39,6.46,39.88,262.0,3.70,1.36,88.87,XY24,NTT,POWER
87,UJJIVANSFB,52.77,53.00,49.36,M-SC,32.29,129222.0,-13257.0,13878.0,0.36,-9.30,10.74,0.44,247.0,-0.96,0.94,52.52,OX40N,NTT,BANKS
71,SBIN,760.30,863.00,-11.51,M-LC,46.88,214756.0,13276.0,13938.0,0.30,6.59,6.49,13.51,77.0,0.95,1.57,20.02,XY25,NTT,BANKS
39,HINDALCO,651.95,761.55,-21.55,H-LC,73.96,107720.0,3408.0,14133.0,-0.27,3.27,13.12,16.81,27.0,0.24,0.79,19.84,X5K,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,36.46,151498.0,585.0,59872.0,0.17,0.39,39.52,40.06,103.0,0.01,1.1,23.92,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-7.22,L-SC,100.00,186372.0,44530.0,12040.0,6.86,31.39,6.46,39.88,262.0,3.70,1.36,88.87,XY24,NTT,POWER
40,HINDUNILVR,2413.81,2723.00,-10.39,X-LC,88.54,280560.0,10213.0,24409.0,4.01,3.78,8.70,12.81,5.0,0.42,2.05,17.34,XY25,NTT,FMCG
56,LAOPALA,369.40,464.00,122.55,H-SC,17.71,70789.0,-30057.0,55881.0,1.71,-29.80,78.94,25.61,136.0,-0.54,0.52,27.67,AR,NTT,CERAMICS
60,NATIONALUM,189.63,247.44,-49.40,H-MC,62.50,101707.0,1203.0,29434.0,1.56,1.20,28.94,30.49,97.0,0.04,0.74,33.99,MH,ATH,METALS
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,13.54,67930.0,-45619.0,85673.0,1.43,-40.18,126.12,35.27,265.0,-0.53,0.50,66.26,XR,NTT,HOTELS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAELXSI,7332.28,7332.00,-21.18,X-MC,4.17,82796.0,-19856.0,19854.0,-3.64,-19.34,23.98,-0.00,59.0,-1.00,0.60,25.86,OX40N,NTT,IT
20,CAMS,3643.00,5250.99,2.54,H-SC,26.04,115240.0,13236.0,31783.0,-1.72,12.98,27.58,44.14,126.0,0.42,0.84,31.32,X40N,ATH,MISC
48,INFY,1461.46,2275.00,-11.34,X-LC,28.12,284252.0,22651.0,122967.0,-1.72,8.66,43.26,55.67,2.0,0.18,2.07,15.29,X40,BTT,IT
83,TCS,3794.03,4998.00,-20.31,X-LC,16.67,259358.0,-36576.0,130483.0,-1.68,-12.36,50.31,31.73,1.0,-0.28,1.89,3.81,X40,BTT,IT
95,WIPRO,243.46,420.00,-5.58,M-LC,59.38,161603.0,10658.0,98804.0,-1.66,7.06,61.14,72.51,71.0,0.11,1.18,11.32,XR,NTT,IT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-221.81,H-SC,93.75,87867.0,-7449.0,7486.0,0.40,-7.82,8.52,0.04,157.0,-1.00,0.64,73.31,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,49.36,M-SC,32.29,129222.0,-13257.0,13878.0,0.36,-9.30,10.74,0.44,247.0,-0.96,0.94,52.52,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.80,M-SC,29.17,151737.0,-19404.0,19346.0,-0.16,-11.34,12.75,-0.03,191.0,-1.00,1.11,9.06,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,87.50,148800.0,-25759.0,25787.0,-0.02,-14.76,17.33,0.02,133.0,-1.00,1.09,29.03,OX40N,NTT,PAINTS
53,KANSAINER,299.63,340.0,-68.02,H-SC,42.71,223335.0,-46332.0,82656.0,-0.38,-17.18,37.01,13.47,137.0,-0.56,1.63,11.78,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-20.31,X-LC,16.67,259358.0,-36576.0,130483.0,-1.68,-12.36,50.31,31.73,1.0,-0.28,1.89,3.81,X40,BTT,IT
48,INFY,1461.46,2275.00,-11.34,X-LC,28.12,284252.0,22651.0,122967.0,-1.72,8.66,43.26,55.67,2.0,0.18,2.07,15.29,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-10.39,X-LC,88.54,280560.0,10213.0,24409.0,4.01,3.78,8.70,12.81,5.0,0.42,2.05,17.34,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,18.13,X-LC,79.17,270756.0,41523.0,25749.0,0.14,18.11,9.51,29.35,10.0,1.61,1.97,28.65,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-6.10,X-LC,12.50,79340.0,-30832.0,119907.0,-0.56,-27.99,151.13,80.85,11.0,-0.26,0.58,16.85,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3151.9,NaN,NaN,97.92,78798.0,44349.0,0.0,0.37,128.74,0.00,128.74,274.0,inf,0.57,20.49,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.2,-7.22,L-SC,100.00,186372.0,44530.0,12040.0,6.86,31.39,6.46,39.88,262.0,3.70,1.36,88.87,XY24,NTT,POWER
59,MEDANTA,1064.65,1486.0,17.58,X-SC,91.67,250591.0,45114.0,36210.0,-0.26,21.96,14.45,39.58,65.0,1.25,1.83,34.28,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,94.40,M-MC,68.75,233469.0,8507.0,159156.0,0.59,3.78,68.17,74.53,189.0,0.05,1.70,35.55,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.97,31.49,H-SC,60.42,134581.0,9284.0,49876.0,-0.02,7.41,37.06,47.22,143.0,0.19,0.98,68.37,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.54,H-SC,26.04,115240.0,13236.0,31783.0,-1.72,12.98,27.58,44.14,126.0,0.42,0.84,31.32,X40N,ATH,MISC
88,UNIONBANK,123.87,163.00,-4.47,M-LC,14.58,165274.0,24434.0,20064.0,1.04,17.35,12.14,31.59,88.0,1.22,1.21,43.57,XY24,NTT,BANKS
15,BLUESTARCO,1646.70,2337.55,-5.12,H-SC,94.79,178060.0,13390.0,55697.0,-0.90,8.13,31.28,41.95,124.0,0.24,1.30,16.20,X40N,ATH,AC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3151.90,NaN,NaN,97.92,78798.0,44349.0,0.0,0.37,128.74,0.00,128.74,274.0,inf,0.57,20.49,AR,ATH,ELECTRICAL
54,KPIGREEN,497.21,731.97,31.49,H-SC,60.42,134581.0,9284.0,49876.0,-0.02,7.41,37.06,47.22,143.0,0.19,0.98,68.37,MH,ATH,POWER
94,WHIRLPOOL,1167.49,2270.00,-40.07,M-SC,48.96,197436.0,25815.0,136251.0,1.11,15.04,69.01,94.43,201.0,0.19,1.44,45.72,XR,NTT,DURABLES
45,INDIAMART,2327.09,4911.36,-46.96,H-SC,82.29,138081.0,14745.0,122215.0,0.74,11.96,88.51,111.05,123.0,0.12,1.01,35.29,AR,ATH,MISC
95,WIPRO,243.46,420.00,-5.58,M-LC,59.38,161603.0,10658.0,98804.0,-1.66,7.06,61.14,72.51,71.0,0.11,1.18,11.32,XR,NTT,IT


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.48,NaN,NaN,71.88,18939.0,-30654.0,0.0,0.14,-61.81,0.00,-61.81,270.0,-inf,0.14,27.01,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,13.54,67930.0,-45619.0,85673.0,1.43,-40.18,126.12,35.27,265.0,-0.53,0.50,66.26,XR,NTT,HOTELS
32,ENRIN,1377.95,3151.90,NaN,NaN,97.92,78798.0,44349.0,0.0,0.37,128.74,0.00,128.74,274.0,inf,0.57,20.49,AR,ATH,ELECTRICAL
6,ASIANTILES,75.41,137.00,7177.78,M-SC,78.12,79840.0,-13970.0,90586.0,-0.39,-14.89,113.46,81.67,245.0,-0.15,0.58,54.61,XR,NTT,CERAMICS
92,VIPIND,488.80,489.00,-221.81,H-SC,93.75,87867.0,-7449.0,7486.0,0.40,-7.82,8.52,0.04,157.0,-1.00,0.64,73.31,OX40N,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-7.22,L-SC,100.00,186372.0,44530.0,12040.0,6.86,31.39,6.46,39.88,262.0,3.70,1.36,88.87,XY24,NTT,POWER
3,ALKYLAMINE,2347.98,4546.37,7.09,X-SC,98.96,99610.0,-1353.0,95885.0,0.29,-1.34,96.26,93.63,67.0,-0.01,0.73,48.37,SR,ATH,CHEMICALS
32,ENRIN,1377.95,3151.90,NaN,NaN,97.92,78798.0,44349.0,0.0,0.37,128.74,0.00,128.74,274.0,inf,0.57,20.49,AR,ATH,ELECTRICAL
65,RELAXO,902.64,1176.00,-27.08,H-SC,96.88,65914.0,-49624.0,84614.0,-0.75,-42.95,128.37,30.28,131.0,-0.59,0.48,28.77,X40N,NTT,FOOTWEAR
5,ASIANPAINT,2961.56,4250.00,-15.98,H-LC,95.83,166872.0,-34514.0,122134.0,0.11,-17.14,73.19,43.51,31.0,-0.28,1.22,16.16,X40,BTT,PAINTS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.96
1,25,41.90
2,50,70.65


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.83
LC    33.61
MC    21.53
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.23
X40      12.77
XY25     12.12
X40N     11.86
XR        9.96
OX40N     8.19
AR        8.18
MH        1.72
X5K       1.41
X200      1.38
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.56
M-SC    15.92
H-LC    13.96
X-LC    12.82
H-MC     9.57
X-MC     7.54
M-LC     5.80
X-SC     4.58
M-MC     4.06
L-SC     2.77
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.30,0.25,33.17
IT,11.85,-11.73,66.59
FINANCE,8.53,-11.43,51.87
BANKS,7.68,-8.39,52.62
MISC,5.87,-19.61,75.72
PAINTS,5.64,-13.00,35.27
ELECTRICAL,5.45,1.36,45.31
HEALTHCARE,4.73,0.74,28.26
AUTO,4.31,-14.38,64.67


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2611731.0,26
XR,1093562.0,13
AR,1029195.0,10
X40,704423.0,10
X40N,536029.0,11
XY25,457310.0,8
OX40N,308243.0,11
SR,178511.0,2
MH,79310.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1970182.0,20
M-SC,1761209.0,21
H-LC,625967.0,12
X-LC,551627.0,8
H-MC,520176.0,8
X-MC,419069.0,6
M-MC,412823.0,3
X-SC,292207.0,5
M-LC,267216.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      888527.0      6
M-SC       XY24      801358.0      7
H-SC       AR        523345.0      3
M-SC       XR        339232.0      4
M-MC       XY24      297707.0      2
H-MC       XY24      295849.0      3
X-LC       X40       289390.0      3
H-SC       XR        217824.0      3
           X40N      206144.0      4
H-LC       X40N      199744.0      4
M-SC       AR        187652.0      2
           OX40N     164460.0      5
H-LC       X40       159759.0      3
X-MC       X40       159483.0      2
M-LC       XY24      154474.0      2
L-SC       XR        148312.0      2
X-SC       XR        120649.0      1
X-LC       AR        119907.0      1
M-SC       XY25      116331.0      1
M-MC       XR        115116.0      1
X-MC       XY25      105345.0      1
M-LC       XR         98804.0      1
X-SC       SR         95885.0      1
X-LC       XY25       90385.0      3
H-SC       OX40N      84466.0      3
H-MC       XY25       83705.0      1
M-SC       SR         82626.0      1
X-MC       XY24       81266.0      1
H-LC       AR         80579.0      1
           X5K        75034.0      2
           X200       70780.0      1
M-SC       X40        69550.0      1
H-MC       AR         59872.0      1
L-SC       AR         57840.0      1
L-MC       XR         53625.0      1
X-MC       X40N       53121.0      1
X-LC       X40N       51945.0      1
H-SC       MH         49876.0      1
L-LC       XY25       47606.0      1
H-LC       XY24       40071.0      1
X-SC       OX40N      39463.0      2
           XY24       36210.0      1
H-MC       MH         29434.0      1
           X40        26241.0      1
           X40N       25075.0      1
X-MC       OX40N      19854.0      1
M-LC       XY25       13938.0      1
L-SC       XY24       12040.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
